In [2]:
from graph_tool import *
from tqdm import tqdm
import csv

In [5]:
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if counter != 10:
            print(row)
            counter +=1
        else: break

10it [00:00, 262.98it/s]

['chr', 'start', 'end', 'geneID', 'REMID', 'regressionCoefficient', 'pValue', 'normModelScore', 'meanDNase1Signal', 'sdDNase1Signal', 'consortium', 'version']
['chr1', '8000', '9999', 'ENSG00000278267', 'REM0000001', '-0.256695911008484', '0.00117467409165201', '0.5224341191833749', '0.02381156626506024', '0.20529241835168552', 'R', '1']
['chr1', '10000', '11999', 'ENSG00000223972', 'REM0000002', '0.282339597095816', '0.00292529340664151', '1.0', '8.482383975903614', '3.3935585369746892', 'R', '1']
['chr1', '16000', '17999', 'ENSG00000223972', 'REM0000003', '0.153983772651832', '0.10480646042063', '0.3866130429076009', '0.7637601445783132', '0.3955437009964319', 'R', '1']
['chr1', '16000', '17999', 'ENSG00000237613', 'REM0000004', '0.71982319723698', '0.000525639660848767', '1.0', '0.7637601626506024', '0.39554369409406526', 'B', '1']
['chr1', '18000', '19999', 'ENSG00000237613', 'REM0000005', '-0.576610369379893', '0.00365802723596124', '0.7430684116746733', '0.4999579939759036', '0.2

In [6]:
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if counter != 10:
            print(row)
            counter +=1
        else: break

10it [00:00, 12614.45it/s]

['REMID', 'sampleID', 'dnase1Log2', 'standDnase1Log2', 'version']
['REM0000001', 'B_C0010KB1', '0', '0.031891038020643886', '1']
['REM0000001', 'B_C0011IB1', '0', '0.04013948313468239', '1']
['REM0000001', 'B_C001UYB4', '0', '0.03228939822426737', '1']
['REM0000001', 'B_C005PS12', '0', '0.04380154958390911', '1']
['REM0000001', 'B_C005VG11', '0', '0.03049425542411712', '1']
['REM0000001', 'B_C0066P12', '0', '0.03271801586763492', '1']
['REM0000001', 'B_C006NSB1', '0', '0.02778819823971882', '1']
['REM0000001', 'B_S001MJ12', '0', '0.017380119422556777', '1']
['REM0000001', 'B_S001S712', '0', '0.04070749185898802', '1']


In [2]:
# get dict with REMID as key and corresponding CREM as value
# dict with CREM as key and contained REMs as value
crem_dict = dict()
rem_crem_dict = dict()
with open('../data/clusterREMs_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "REMID":
            crem_dict[row[0]] = [row[1], row[3], row[4]]
            if rem_crem_dict.get(row[1]) == None:
                rem_crem_dict[row[1]] = [row[0]]
            else:
                rem_crem_dict[row[1]].append(row[0])

1264526it [00:05, 246296.52it/s]


In [3]:
# get dict with GENE name as key and start and end as value
gene_dict = dict()
with open('../data/GeneAnnotation_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "chr":
            gene_dict[row[3]] = [int(row[1]), int(row[2]), row[0]]

58175it [00:00, 386556.33it/s]


In [4]:
sample_celltype_dict = dict()
ID_celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            # load dict with sample and corresponding celltype
            sample_celltype_dict[row[0]] = [row[1], row[2]]
            
            # load dict with celltype ID and corresponding celltype
            ID_celltype_dict[row[1]] = row[2]

# print(sample_celltype_dict)
# print(ID_celltype_dict)

In [5]:
# dict for genes with their expression values in each sample for both data sets (Roadmap and Blueprint)
gene_expr_dict = dict()
with open('../data/GeneExpressionRoadmap_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs
                
with open('../data/GeneExpressionBlueprint_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs


6399141it [00:09, 682765.02it/s]
3257745it [00:04, 689414.61it/s]


In [7]:
# rem_dict{celltypes : {samples: {REMs: score}}}
# dict to map celltype to samples and the included REMs with their scores

rem_dict = dict()
with open('../data/REMActivity_1.csv') as file_1:
    reader_1 = csv.reader(file_1)
    for r in tqdm(reader_1):
        rem = r[0] 
        sample = r[1] 
        score = r[3] 
        
        if sample_celltype_dict.get(sample) == None:       # if sample not 
            continue
        else:
            celltype = sample_celltype_dict.get(sample)[0]
            
        if rem_dict.get(celltype) == None:
            rem_dict[celltype] = {sample: {rem: score}}
            
        elif rem_dict[celltype].get(sample) == None:
            rem_dict.get(celltype)[sample] = {rem: score}
            
        else:
            rem_dict[celltype].get(sample)[rem] = score
            

399206927it [16:12, 410580.70it/s]


In [8]:
# create template graph to fill with celltype specific values afterwards
# Graph contains gens and rems and their edges buth with no values

g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
start = g.new_vp("int")
g.vp.start = start
end = g.new_vp("int")
g.vp.end = end
rem = g.new_ep("string")
g.ep.rem = rem
chr = g.new_vp("string")
g.vp.chr = chr
celltypeID = g.new_ep("string")
g.edge_properties["celltypeID"] = celltypeID
celltype = g.new_ep("string")
g.edge_properties["celltype"] = celltype

done_proteins_dict = {}
done_crems = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        rem_name = row[4]
        gen = row[3]
        crem = crem_dict.get(rem_name)
        if crem != None:
            crem = crem[0]
        if row[1] != "start":
            
            if crem in done_crems:
                v1 = g.vertex(done_crems.get(crem))
            elif crem != None:
                v1 = g.add_vertex()
                g.vp.name[v1] = crem
                g.vp.type[v1] = "REM"
                g.vp.start[v1] = crem_dict.get(rem_name)[1]
                g.vp.end[v1] = crem_dict.get(rem_name)[2]
                done_crems[crem] = int(v1)
            else:
                v1 = g.add_vertex()
                g.vp.name[v1] = rem_name
                g.vp.type[v1] = "REM"
                g.vp.start[v1] = int(row[1])
                g.vp.end[v1] = int(row[2])

            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                g.vp.start[v2] = gene_dict.get(gen)[0]
                g.vp.end[v2] = gene_dict.get(gen)[1]
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))

            e = g.add_edge(v1, v2)
            g.ep.rem[e] = rem_name
                
g.save(f"../data/graph-tool/template.gt")

2404862it [00:59, 40344.96it/s]


In [9]:
# load template graph and fill with celltype edge values by iterating over samples in celltype

graph = load_graph(f"../data/graph-tool/template.gt")

for celltype_id in rem_dict:
#     if celltype_id not in ["CTID_0000006"]:
#         continue
    print(celltype_id)
    
    g = Graph(graph)
    samples = rem_dict.get(celltype_id)
    celltype_name = ID_celltype_dict.get(celltype_id)
    
    for sample in samples:
        sampleID = g.new_ep("double")
        g.edge_properties[sample] = sampleID
        sample_g = g.new_vp("double")
        g.vertex_properties[sample]= sample_g
    
        for e in tqdm(g.edges()):
            rem = g.vp.name[e.source()]
            g.ep.celltypeID[e] = celltype_id
            g.ep.celltype[e] = celltype_name
            
            gene = g.vp.name[e.target()]                      # new part, adding gene expr for specific sample
            expr = gene_expr_dict.get(gene).get(sample) 
            g.vp[sample][e.target()] = expr
            
#             if rem.startswith("C"):    #  mean val for crems
#                 mean_expr = sum(float(samples.get(sample).get(r)) for r in rem_crem_dict.get(rem))/len(rem_crem_dict.get(rem))
#                 g.ep[sample][e] = mean_expr
#             else:
#                 g.ep[sample][e] = samples.get(sample).get(rem) 
            
            rem_edge_name = g.ep.rem[e]
            g.ep[sample][e] = samples.get(sample).get(rem_edge_name)
            
    g.save(f"../data/graph-tool/{celltype_id}.gt")
#     break

CTID_0000006


2404861it [03:09, 12687.18it/s]
2404861it [03:07, 12812.09it/s]
2404861it [03:08, 12777.27it/s]


CTID_0000027


2404861it [03:23, 11805.83it/s]


CTID_0000033


...

2404861it [03:06, 12875.41it/s]


CTID_0000065


2404861it [03:14, 12386.86it/s]


CTID_0000063


2404861it [03:05, 12996.53it/s]


In [ ]:
# check if all edges are the same in all samples
d = dict()
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if row[0] != "REMID":
            if d.get(row[1]) != None:
                d[row[1]].add(row[0])
            else:
                d[row[1]] = {row[0]}
        
check = list(d.values())[0]
for k in d.keys():
#     print(k)
    if  len(check.difference(d.get(k))) != 0:
        print(k)